In [ ]:
#Finta signals with ML

In [60]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests
from finta import TA

In [61]:
# Load environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=1095)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PK6JCY12U8K9OYRH7Z6Q", "u4tivaUmWg6VrGYLQnGMZ2PI9RDaWVkMkh9WlMhL", ALPACA_API_BASE_URL, api_version="v2")

In [62]:
# Set the ticker symbol and the number of shares to buy
ticker = "LTCUSD"
number_of_coins = 2000

# Make API call
LTC_prices = api.get_crypto_bars(ticker, '1day',
                         start=_3YearsAgo.isoformat(),
                         end=None,
                         limit=None).df

In [92]:
#Filter data by exchange
LTC_prices = LTC_prices[LTC_prices.exchange == 'CBSE']

In [64]:
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [65]:
# Create a signals_df DataFrame that is a copy of the ixn_df Dataframe
signals_df = LTC_prices.copy()

In [66]:
signals_df.drop(columns=['exchange','volume','trade_count','vwap'],inplace=True)

In [67]:
signals_df.tail()

,open,high,low,close
timestamp,,,,
2022-10-22 05:00:00+00:00,51.53,52.50,51.48,52.20
2022-10-23 05:00:00+00:00,52.20,54.22,51.83,53.49
2022-10-24 05:00:00+00:00,53.46,53.53,52.11,52.78
2022-10-25 05:00:00+00:00,52.79,56.84,52.27,56.26
2022-10-26 05:00:00+00:00,56.26,57.63,55.75,56.19


In [68]:
# Set the short window and long windows
fast_window = 15
slow_window = 50

# Add the SMA technical indicators for the short and long windows
signals_df["fast"] = TA.DEMA(signals_df, fast_window)
signals_df["slow"] = TA.DEMA(signals_df, slow_window)

In [69]:
signals_df[["fast","slow"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [70]:
signals_df.head()

,open,high,low,close,fast,slow
timestamp,,,,,,
2019-10-28 05:00:00+00:00,59.46,60.10,57.64,57.83,57.830000,57.830000
2019-10-29 05:00:00+00:00,59.85,61.19,58.89,59.23,58.925111,58.893860
2019-10-30 05:00:00+00:00,59.22,59.68,56.82,57.96,58.414532,58.437755
2019-10-31 05:00:00+00:00,58.39,58.39,57.52,57.94,58.186203,58.227514
2019-11-01 05:00:00+00:00,57.96,59.00,56.66,58.34,58.248303,58.269643


In [71]:
# Set the Signal column
signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the fast window is greater than (or crosses over) the slow Window
# where 0 is when the fast window is under the slow window
signals_df["Signal"][fast_window:] = np.where(
    signals_df["fast"][fast_window:] > signals_df["slow"][fast_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.iloc[95:105, :]

,open,high,low,close,fast,slow,Signal,Entry/Exit
timestamp,,,,,,,,
2020-01-31 06:00:00+00:00,66.60,69.80,65.22,67.96,63.566364,56.433947,1.0,0.0
2020-02-01 06:00:00+00:00,70.69,70.72,67.64,69.18,65.384518,57.582139,1.0,0.0
2020-02-02 06:00:00+00:00,69.23,73.33,68.92,69.82,66.985845,58.708135,1.0,0.0
2020-02-03 06:00:00+00:00,69.59,70.46,68.18,68.66,67.984269,59.671909,1.0,0.0
2020-02-04 06:00:00+00:00,67.83,69.32,67.43,69.10,68.862370,60.608312,1.0,0.0
2020-02-05 06:00:00+00:00,72.59,74.00,70.92,73.49,70.567286,61.827805,1.0,0.0
2020-02-06 06:00:00+00:00,73.53,75.00,71.11,73.45,71.908905,62.966556,1.0,0.0
2020-02-07 06:00:00+00:00,74.28,74.54,71.53,73.49,72.969535,64.035376,1.0,0.0
2020-02-08 06:00:00+00:00,76.66,77.95,76.65,77.48,74.724870,65.346644,1.0,0.0


In [72]:
signals_df["Signal"].value_counts()

1.0    598
0.0    497
Name: Signal, dtype: int64

In [73]:
signals_df["Entry/Exit"].value_counts()

 0.0    1053
 1.0      21
-1.0      20
Name: Entry/Exit, dtype: int64

In [74]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["fast", "slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot


:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [75]:
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [76]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['fast', 'slow']].shift().dropna()

# Review the DataFrame
X.head()

,fast,slow
timestamp,,
2019-10-29 05:00:00+00:00,57.830000,57.830000
2019-10-30 05:00:00+00:00,58.925111,58.893860
2019-10-31 05:00:00+00:00,58.414532,58.437755
2019-11-01 05:00:00+00:00,58.186203,58.227514
2019-11-02 05:00:00+00:00,58.248303,58.269643


In [77]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

1.0    598
0.0    497
Name: Signal, dtype: int64

In [78]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-10-29 05:00:00+00:00


In [79]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end)

2021-10-29 05:00:00+00:00


In [80]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,fast,slow
timestamp,,
2019-10-29 05:00:00+00:00,57.830000,57.830000
2019-10-30 05:00:00+00:00,58.925111,58.893860
2019-10-31 05:00:00+00:00,58.414532,58.437755
2019-11-01 05:00:00+00:00,58.186203,58.227514
2019-11-02 05:00:00+00:00,58.248303,58.269643


,fast,slow
timestamp,,
2021-10-25 05:00:00+00:00,197.830384,186.282052
2021-10-26 05:00:00+00:00,198.311666,187.310804
2021-10-27 05:00:00+00:00,200.304028,188.818821
2021-10-28 05:00:00+00:00,196.853274,188.586369
2021-10-29 05:00:00+00:00,196.590458,189.219901


In [81]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,fast,slow
timestamp,,
2021-10-30 05:00:00+00:00,196.167357,189.757076
2021-10-31 05:00:00+00:00,195.945493,190.311066
2021-11-01 05:00:00+00:00,195.421167,190.726262
2021-11-02 05:00:00+00:00,196.256275,191.536651
2021-11-03 05:00:00+00:00,197.736962,192.565331


,fast,slow
timestamp,,
2022-10-22 05:00:00+00:00,51.059520,51.552972
2022-10-23 05:00:00+00:00,51.228089,51.510660
2022-10-24 05:00:00+00:00,51.674679,51.571853
2022-10-25 05:00:00+00:00,51.878557,51.576697
2022-10-26 05:00:00+00:00,52.864362,51.850608


In [82]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [83]:
from sklearn import svm
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [84]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       1.00      0.46      0.63       185
         1.0       0.64      1.00      0.78       177

    accuracy                           0.73       362
   macro avg       0.82      0.73      0.71       362
weighted avg       0.82      0.73      0.71       362



In [85]:
# Use the pct_change function to generate returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Create an empty predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['SVM Returns'] = signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,SVM Returns
timestamp,,,
2021-10-30 05:00:00+00:00,1.0,0.003697,0.003697
2021-10-31 05:00:00+00:00,1.0,-0.006744,-0.006744
2021-11-01 05:00:00+00:00,1.0,0.028936,0.028936
2021-11-02 05:00:00+00:00,1.0,0.017970,0.017970
2021-11-03 05:00:00+00:00,1.0,0.017802,0.017802


,Predicted,Actual Returns,SVM Returns
timestamp,,,
2022-10-22 05:00:00+00:00,1.0,0.012806,0.012806
2022-10-23 05:00:00+00:00,1.0,0.024713,0.024713
2022-10-24 05:00:00+00:00,1.0,-0.013274,-0.013274
2022-10-25 05:00:00+00:00,1.0,0.065934,0.065934
2022-10-26 05:00:00+00:00,1.0,-0.001244,-0.001244


In [86]:
# Plot the actual returns versus the SVM strategy returns 
(1 + predictions_df[["Actual Returns", "SVM Returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [87]:
# Import a new classifier from SKLearn
from sklearn import tree

# Initiate the model instance
dtc_model=tree.DecisionTreeClassifier()

In [88]:
# Fit the model using the training data
dtc_model = dtc_model.fit(X_train_scaled,y_train)

# Use the testing dataset to generate the predictions for the new model
dtc_pred = dtc_model.predict(X_test_scaled)
# Review the model's predicted values
dtc_pred[:10]

array([1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [89]:
# Use a classification report to evaluate the model using the predictions and testing data
dtc_testing_report = classification_report(y_test, dtc_pred)

# Print the classification report
print(dtc_testing_report)

              precision    recall  f1-score   support

         0.0       0.85      0.56      0.68       185
         1.0       0.66      0.89      0.76       177

    accuracy                           0.72       362
   macro avg       0.75      0.73      0.72       362
weighted avg       0.76      0.72      0.72       362



In [90]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
final_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
final_df["Predicted"]=dtc_pred

# Add the actual returns to the DataFrame
final_df["Actual Returns"]=signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
final_df["DTC Returns"]=final_df["Actual Returns"]*final_df["Predicted"]

# Review the DataFrame
display(final_df.head())
display(final_df.tail())

,Predicted,Actual Returns,DTC Returns
timestamp,,,
2021-10-30 05:00:00+00:00,1.0,0.003697,0.003697
2021-10-31 05:00:00+00:00,1.0,-0.006744,-0.006744
2021-11-01 05:00:00+00:00,1.0,0.028936,0.028936
2021-11-02 05:00:00+00:00,1.0,0.017970,0.017970
2021-11-03 05:00:00+00:00,1.0,0.017802,0.017802


,Predicted,Actual Returns,DTC Returns
timestamp,,,
2022-10-22 05:00:00+00:00,0.0,0.012806,0.0
2022-10-23 05:00:00+00:00,0.0,0.024713,0.0
2022-10-24 05:00:00+00:00,0.0,-0.013274,-0.0
2022-10-25 05:00:00+00:00,0.0,0.065934,0.0
2022-10-26 05:00:00+00:00,0.0,-0.001244,-0.0


In [91]:
# Plot the actual returns versus the strategy returns
(1 + final_df[["Actual Returns", "DTC Returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)